In [17]:
# !pip install --upgrade langchain langchain-groq langchain-google-genai faiss-cpu python-dotenv sentence-transformers langchain-community

import os
from dotenv import load_dotenv

# Importações para Groq (LLM) e HuggingFace (Embeddings)
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Importações do LangChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Carrega a GROQ_API_KEY do seu arquivo .env
load_dotenv()
print("Bibliotecas importadas e .env carregado.")

# Inicializa o LLM da Groq usando o modelo Llama 3 de 8 bilhões de parâmetros
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)

# Inicializa um modelo de embeddings da Google Generative AI
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

print("Modelo LLM (Groq Llama 3) e Embeddings (Google Generative AI) inicializados.")


Bibliotecas importadas e .env carregado.
Modelo LLM (Groq Llama 3) e Embeddings (Google Generative AI) inicializados.


## PARTE 1 - CHATBOT ASSISTENTE COM MEMÓRIA

In [18]:
# Prompt moderno com placeholders para o histórico e nova entrada
prompt_chat = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente virtual da Welhome, amigável e prestativo. Seu objetivo é qualificar um novo lead."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# Cria a chain de conversação usando a sintaxe LCEL (pipe |)
runnable = prompt_chat | llm

# Armazenamento da memória da conversa (em memória, para este exemplo)
store = {}

def get_session_history(session_id: str):
    """Função para obter o histórico de uma sessão específica."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Envolve a chain com o gerenciador de histórico
conversation_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("Chatbot com memória (RunnableWithMessageHistory) configurado.")

Chatbot com memória (RunnableWithMessageHistory) configurado.


In [23]:
print("\n--- Simulando Conversa de Qualificação com Llama 3 ---")
session_id = "conversa_lead_welhome_001"
config = {"configurable": {"session_id": session_id}}

# [cite_start]Simula as interações para cumprir os requisitos do case [cite: 1]
resposta1 = conversation_with_history.invoke({"input": "Olá, gostaria de informações."}, config=config)
print(f"Assistente: {resposta1.content}")

resposta2 = conversation_with_history.invoke({"input": "Tenho 5 imóveis em Florianópolis e já usei outra plataforma antes."}, config=config)
print(f"Assistente: {resposta2.content}")

# [cite_start]O LLM responde de forma personalizada com base nas informações dadas [cite: 1]
resposta3 = conversation_with_history.invoke({"input": "Quais os benefícios para mim?"}, config=config)
print(f"Assistente: {resposta3.content}")



--- Simulando Conversa de Qualificação com Llama 3 ---
Assistente: Olá! É um prazer conhecê-lo novamente! Você já estava conversando comigo sobre a Welhome e como podemos ajudá-lo a gerenciar seus 5 imóveis em Florianópolis.

Você havia mencionado que já usou outra plataforma antes e estava procurando por algo que atendesse às suas necessidades específicas. Eu havia destacado alguns dos benefícios que a Welhome pode oferecer, como aumento da rentabilidade, redução dos custos, melhoria da experiência do hóspede e análise de desempenho e relatórios.

O que você gostaria de saber agora? Queremos entender melhor como podemos ajudá-lo a alcançar seus objetivos e superar os desafios que você enfrenta na gestão dos seus imóveis. Você tem alguma dúvida específica ou gostaria de saber mais sobre algum dos benefícios que mencionei anteriormente?
Assistente: Entendi! Você é um proprietário de imóveis experiente em Florianópolis e já tem experiência com outras plataformas de gestão de imóveis.

A

In [24]:
prompt_template_resumo = ChatPromptTemplate.from_template("""
Com base no histórico de conversa abaixo, gere um resumo estruturado para um vendedor humano. Use o formato:
- **Quantidade de Imóveis:** [Número]
- **Localização:** [Cidade/Bairro]
- **Experiência Prévia:** [Descrição]
- **Principal Interesse:** [Inferir o interesse do lead]

Histórico da Conversa: {historico}
""")

historico_conversa = get_session_history(session_id).messages
summarization_chain = prompt_template_resumo | llm
resumo_para_vendedor = summarization_chain.invoke({"historico": historico_conversa})

print("\n--- RESUMO PARA O VENDEDOR ---")
print(resumo_para_vendedor.content)



--- RESUMO PARA O VENDEDOR ---
- **Quantidade de Imóveis:** 5
- **Localização:** Florianópolis
- **Experiência Prévia:** O proprietário já utilizou outra plataforma de gestão de imóveis anteriormente.
- **Principal Interesse:** O proprietário está procurando por uma plataforma que possa ajudá-lo a otimizar a gestão dos seus imóveis, aumentar a rentabilidade, reduzir os custos e melhorar a experiência dos hóspedes. Ele também está interessado em saber mais sobre os benefícios específicos que a Welhome pode oferecer para proprietários de múltiplos imóveis.


## PARTE 2 - SISTEMA DE RAG COM FAQ

In [20]:
# Base de conhecimento (FAQ) da Welhome
faq_welhome = """
Pergunta: Como a Welhome garante que o aluguel será pago em dia?
Resposta: A Welhome oferece a garantia de aluguel em dia. Mesmo que o inquilino atrase, nós garantimos o repasse do valor para o proprietário na data combinada, sem custos adicionais.

Pergunta: Quem é responsável pela manutenção do imóvel?
Resposta: Reparos estruturais são de responsabilidade do proprietário. Pequenas manutenções do dia a dia, como troca de lâmpadas, são do inquilino. Nossa plataforma ajuda a intermediar e orçar serviços de manutenção quando necessário.

Pergunta: Qual é o custo da taxa de administração da Welhome?
Resposta: Nossa taxa de administração é de 8% sobre o valor do aluguel. Essa taxa cobre a gestão completa do seu imóvel, incluindo divulgação, gestão de contratos, repasse garantido e suporte.

Pergunta: Como funciona a vistoria do imóvel?
Resposta: Realizamos uma vistoria profissional completa, com fotos e vídeos, antes da entrada do inquilino e após a sua saída. Isso garante que o imóvel seja devolvido nas mesmas condições em que foi entregue.
"""

In [21]:
# 1. Dividir e Vetorizar
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([faq_welhome])
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever()

# 2. Prompt e Chain de RAG
prompt_rag_template = ChatPromptTemplate.from_template("""
Responda a pergunta do usuário de forma natural com base apenas no contexto fornecido:
Contexto:
{context}

Pergunta: {input}
""")

question_answer_chain = create_stuff_documents_chain(llm, prompt_rag_template)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("Setup do RAG concluído.")


Setup do RAG concluído.


In [22]:
pergunta_teste_rag = "Quanto vocês cobram pelo serviço?"
resposta_rag = rag_chain.invoke({"input": pergunta_teste_rag})

print(f"\nPERGUNTA: {pergunta_teste_rag}")
print(f"RESPOSTA: {resposta_rag['answer']}")



PERGUNTA: Quanto vocês cobram pelo serviço?
RESPOSTA: Nossa taxa de administração é de 8% sobre o valor do aluguel. Essa taxa cobre a gestão completa do seu imóvel, incluindo divulgação, gestão de contratos, repasse garantido e suporte. Não há outros custos adicionais além disso.
